In [1]:
import sys
import os
import math
import euclid
import mesh
import geomgen
import fbx

In [2]:
sys.path = sys.path + [os.curdir + os.sep + "fbx2018_1"]    

In [3]:
 from FbxCommon import *

In [4]:
def create_sdk_manager():
    result = FbxManager.Create()
    return result

def create_scene(manager, name = "FbxScene"):
    ios = FbxIOSettings.Create(manager, IOSROOT)
    manager.SetIOSettings(ios)
    scene = FbxScene.Create(manager, name)
    return scene



In [5]:
def load_scene(manager, scene, file_name):
    importer = FbxImporter.Create(manager, "")    
    result = importer.Initialize(file_name, -1, manager.GetIOSettings())
    if not result:
        return False    
    if importer.IsFBX():
        manager.GetIOSettings().SetBoolProp(EXP_FBX_MATERIAL, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_TEXTURE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_EMBEDDED, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_SHAPE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GOBO, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_ANIMATION, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GLOBAL_SETTINGS, True)
    
    result = importer.Import(scene)
    importer.Destroy()
    return result

In [6]:
# Prepare the FBX SDK.
lSdkManager, lScene = InitializeSdkObjects()

In [7]:
lResult = LoadScene(lSdkManager, lScene, "Track.fbx")

In [8]:
 lRootNode = lScene.GetRootNode()

In [9]:
def get_node_geometric_transform(node):
    t = node.GetGeometricTransformation()

In [10]:
def print_node_info(node):
    print node.GetName()
    print "="
    
    print "="

In [11]:
def walk_nodes_aux(node):
    for i in range(node.GetChildCount()):
        print node.GetChild(i).GetName()
        walk_nodes_aux(node.GetChild(i))
        
def walk_nodes(scene):
    root = scene.GetRootNode()
    walk_nodes_aux(root)

In [12]:
walk_nodes(lScene)

Track_Node


In [13]:
lScene.Destroy()

In [14]:
scene = create_scene(lSdkManager, "Generic")

In [15]:
load_scene(lSdkManager, scene, "GenericMan.fbx" )

True

In [76]:
root_node = scene.GetRootNode()

In [77]:
walk_nodes(scene)

Character
Root
Waist
Left Hip
Left Knee
Left Foot
Right Hip
Right Knee
Right Foot
Middle Spine
Neck
Left Arm
Left Elbow
Left Hand
Head
Right Arm
Right Elbow
Right Hand


In [18]:
FbxTransform.__doc__

'FbxTransform()\nFbxTransform(FbxTransform)'

In [21]:
root_node.EvaluateGlobalTransform(FbxTime(0.0))

In [22]:
FbxTime(0.0)

In [23]:
gt = root_node.GetGeometricTranslation(FbxNode.eSourcePivot)
gr = root_node.GetGeometricRotation(FbxNode.eSourcePivot)
gs = root_node.GetGeometricScaling(FbxNode.eSourcePivot)

In [24]:
print gt, gr, gs

fbx.FbxVector4(0.000000, 0.000000, 0.000000, 1.000000) fbx.FbxVector4(0.000000, 0.000000, 0.000000, 1.000000) fbx.FbxVector4(1.000000, 1.000000, 1.000000, 1.000000)


In [25]:
FbxAMatrix(gt, gr, gs)

In [26]:
def get_geometry_transform(node):
    gt = node.GetGeometricTranslation(FbxNode.eSourcePivot)
    gr = node.GetGeometricRotation(FbxNode.eSourcePivot)
    gs = node.GetGeometricScaling(FbxNode.eSourcePivot)
    return FbxAMatrix(gt, gr, gs)

In [27]:
def get_global_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateGlobalTransform(t)

In [32]:
def get_local_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateLocalTransform(t)

In [33]:
print get_global_transform(root_node, 0.0)

In [34]:
def get_node_transform(node, time = 0.0):
    result = get_global_transform(node, time)
    if (node.GetNodeAttribute()):
        gt = get_geometry_transform(node)
        result = result * gt
    return result

In [36]:
print get_node_transform(root_node, 0.0)

In [59]:
def get_node_local_transform(node, time = 0.0):
    transform = get_node_transform(node, time)
    parent = node.GetParent()
    if parent:
        parent_transform = get_node_transform(parent, time)
        inverse_parent_transform = parent_transform.Inverse()
        transform = transform * inverse_parent_transform
    return transform

In [50]:
print get_node_local_transform(root_node, 0.0)

In [52]:
root_node.GetChildCount()

2

In [53]:
child_node = root_node.GetChild(1)

In [60]:
print get_node_local_transform(child_node, 0.0)

In [61]:
def get_node_animated_transform(node, time = 0.0):
    global_transform = node.EvaluateGlobalTransform(FbxTime(time))
    geometry_transform = get_geometry_transform(node)
    return global_transform * geometry_transform
                                                   

In [62]:
print get_node_animated_transform(root_node, 0.5)

In [63]:
root_node??

In [64]:
scene = root_node.GetScene()

In [65]:
scene??

In [66]:
print scene.FindMember.__doc__

FbxCollection.FindMember(FbxClassId, str) -> FbxObject


In [67]:
print scene.FillAnimStackNameArray.__doc__

FbxDocument.FillAnimStackNameArray(FbxStringArray)


In [68]:
string_array = fbx.FbxStringArray()

In [69]:
classid = FbxAnimStack.ClassId

In [70]:
print classid

In [71]:
print scene.GetAnimationEvaluator()

In [72]:
idle_anim_stack = scene.FindMember(FbxAnimStack.ClassId, "Idle")

In [73]:
print idle_anim_stack.GetReferenceTimeSpan().GetStart().GetSecondDouble()

0.0


In [74]:
def get_anim_stacks(scene):
    anim_stack_classid = FbxAnimStack.ClassId
    stack_names = fbx.FbxStringArray()
    scene.FillAnimStackNameArray(stack_names)
    num_stacks = stack_names.GetCount()
    for stack_index in range(0,num_stacks):
        anim_name = stack_names[stack_index]
        if anim_name == None:
            continue
        anim_stack = scene.FindMember(anim_stack_classid, anim_name.Buffer())
        reference_time_span = anim_stack.GetReferenceTimeSpan()
        local_time_span = anim_stack.GetLocalTimeSpan()
        print "Anim Stack ", anim_name
        print "Reference Start, Stop ", anim_stack.GetReferenceTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetReferenceTimeSpan().GetStop().GetSecondDouble()
        print "Local Start, Stop ", anim_stack.GetLocalTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetLocalTimeSpan().GetStop().GetSecondDouble()
        
        

In [75]:
get_anim_stacks(scene)

Anim Stack  Idle
Reference Start, Stop  0.0 1.50000007823
Local Start, Stop  0.0 1.50000007823
Anim Stack  Walk
Reference Start, Stop  0.0 1.80000009386
Local Start, Stop  0.0 1.80000009386
Anim Stack  Jump
Reference Start, Stop  0.0 1.80000009386
Local Start, Stop  0.0 1.80000009386
Anim Stack  Crouch
Reference Start, Stop  0.0 1.80000009386
Local Start, Stop  0.0 1.80000009386


In [78]:
print root_node.GetNodeAttribute()


None


In [83]:
print FbxLight.ClassId

In [81]:
print root_node.GetChild(0).GetNodeAttribute().ClassId

In [102]:
def classify_node(node):
    kinds = { FbxNodeAttribute.eMarker: "Marker",   FbxNodeAttribute.eSkeleton: "Bone", FbxNodeAttribute.eMesh: "Mesh", FbxNodeAttribute.eNurbs: "Nurbs", FbxNodeAttribute.ePatch: "Patch", FbxNodeAttribute.eCamera: "Camera" }
    result = None
    result = node.GetNodeAttribute()
    if result:
        node_kind = result.GetAttributeType()
        if kinds.has_key(node_kind):
            print "Is A ", kinds[node_kind]
        else:
            print "Unknown Type ", node_kind
    return result

In [103]:
def walk_nodes_aux(node,fn):
    for i in range(node.GetChildCount()):
        print node.GetChild(i).GetName()
        fn(node.GetChild(i))
        walk_nodes_aux(node.GetChild(i), fn)
        
def walk_nodes(scene, fn):
    root = scene.GetRootNode()
    fn(root)
    walk_nodes_aux(root, fn)

In [104]:
walk_nodes(root_node.GetScene(), classify_node)

Character
Is A  Mesh
Root
Is A  Bone
Waist
Is A  Bone
Left Hip
Is A  Bone
Left Knee
Is A  Bone
Left Foot
Is A  Bone
Right Hip
Is A  Bone
Right Knee
Is A  Bone
Right Foot
Is A  Bone
Middle Spine
Is A  Bone
Neck
Is A  Bone
Left Arm
Is A  Bone
Left Elbow
Is A  Bone
Left Hand
Is A  Bone
Head
Is A  Bone
Right Arm
Is A  Bone
Right Elbow
Is A  Bone
Right Hand
Is A  Bone
